# Segmenting and Clustering Neighborhoods in Toronto

### Read data from webpage

In [25]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

#Read data from Wikipedia page
toronto_rawdata = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

#assign the table data to a new data frame
dfToronto = toronto_rawdata[0]

#Assignment requires three columns: PostalCode, Borough, and Neighborhood. Rename the column "Postcal Code" to "PostcalCode"
dfToronto.rename(columns = {'Postal Code':'PostalCode'}, inplace = True) 

#dfToronto

#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
#Filter out the not assigned borough
dfToronto_AssignedBorough = dfToronto[dfToronto['Borough']!='Not assigned']

#merge Neighbourhood with same PostcalCode
dtToronto_Merged = dfToronto_AssignedBorough.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
#CHeck if there is any not assigned neighborhoud, The merge happened first, a not-assigned neighborhood could be merged
dtToronto_Merged.loc[(dtToronto_Merged['Neighbourhood'].str.contains('Not assigned')),'Neighbourhood'] = dtToronto_Merged.loc[(dtToronto_Merged['Neighbourhood'].str.contains('Not assigned')),'Borough']


In [27]:
#use the .shape method to print the number of rows of your dataframe.
dtToronto_Merged.shape

(103, 3)